In [1]:
!python -V

Python 3.12.9


In [2]:
import pandas as pd

In [3]:
import pickle
from pathlib import Path

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment-homework")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1749515419823, experiment_id='1', last_update_time=1749515419823, lifecycle_stage='active', name='nyc-taxi-experiment-homework', tags={}>

In [6]:
yellow_march2023 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet")
print(f"Number of records in df_train: {yellow_march2023.shape}")

Number of records in df_train: (3403766, 19)


In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [9]:
print(f"Number of records in df_train: {df_train.shape}")

Number of records in df_train: (3316216, 20)


In [10]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values

In [12]:
models_folder = Path('models')
models_folder.mkdir(exist_ok=True)

In [13]:
mlflow.sklearn.autolog()

with mlflow.start_run():

    # mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet")

    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    intercept = lr_model.intercept_
    
    mlflow.log_metric("intercept", intercept)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.sklearn.log_model(lr_model, artifact_path="models_mlflow")

2025/06/10 01:22:43 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: Unable to allocate 12.8 GiB for an array with shape (3316216, 519) and data type float64
2025/06/10 01:22:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run bittersweet-pig-958 at: http://localhost:5000/#/experiments/1/runs/1493174b898345e2921bfa3b84a12cd3
🧪 View experiment at: http://localhost:5000/#/experiments/1
